# Preprocessing

First take a look at the data

In [52]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.graph_objects as go
import sklearn
import sklearn.compose
from pandas.api.types import is_numeric_dtype

In [6]:
csv_data = pd.read_csv('immo_dev_data.csv')

# display data transposed, easier to see all columns
csv_data.head().T

,0,1,2,3,4
Id,7135329,7170979,7172246,7172252,7172733
AreaLiving,140,143,160,351,400
AreaProperty,501,277,712,496,1800
BuiltYear,2016,2004,1945,2016,1975
FloorNumber,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
gde_workers_sector3,358,2787,1138,17,701
gde_workers_total,537,5041,2999,39,732
location_has_street,0,1,0,0,0
location_is_complete,0,1,0,0,0


Die folgenden Attribute müssen überprüft werden:
- Id
    - des Objects
- Last update??
- SourceId
    - der Quelle/Website
- Name
    - Name der Website
- Adresse?
    - Strasse und Nummer oder Reicht ZIP?


# 

In [9]:
csv_data_head = csv_data.head()
pd.get_dummies(csv
_data_head)

,Id,AreaLiving,AreaProperty,BuiltYear,FloorNumber,ForestDensityL,ForestDensityM,ForestDensityS,HouseObject,Latitude,...,Locality_Saint-Oyens,Name_Homegate,Name_Immoscout,Name_NabHome,StateShort_AG,StateShort_GE,StateShort_JU,StateShort_SO,StateShort_VD,StreetAndNr_Im Hag 3
0,7135329,140.0,501.0,2016,NaN,0.418964,0.555985,0.730714,True,47.322502,...,0,0,1,0,0,0,0,1,0,0
1,7170979,143.0,277.0,2004,NaN,0.033259,0.074061,0.076468,True,47.393546,...,0,0,0,1,1,0,0,0,0,1
2,7172246,160.0,712.0,1945,NaN,0.000000,0.000000,0.000000,True,47.336254,...,0,1,0,0,0,0,1,0,0,0
3,7172252,351.0,496.0,2016,NaN,0.037575,0.000000,0.000000,True,46.498817,...,1,1,0,0,0,0,0,0,1,0
4,7172733,400.0,1800.0,1975,NaN,0.095162,0.097193,0.153314,True,46.240700,...,0,1,0,0,0,1,0,0,0,0


# One Hot Encoding

In [65]:
enc_data = csv_data.drop(columns = ['StreetAndNr'])

In [115]:
def encode_df(_df, enc):
    '''
    This function encodes (one hot encoding) the categorical columns.

    Takes:
    - _df: pandas.DataFrame
    - enc: sklearn.preprocessing.OneHotEncoder(handle_unknown='ignore') object

    returns:
    - encoded df as numpy.array
    '''

    df = _df.copy()
    df_obj = df.select_dtypes(include = [object])
    df = df.drop(columns = df_obj.columns)
    print(df_obj.shape)
    print(df.to_numpy().shape)
    print(enc.fit(df_obj))
    return np.concatenate((df.to_numpy(), enc.transform(df_obj)), axis = 0)

In [116]:
enc = sklearn.preprocessing.OneHotEncoder(handle_unknown='ignore')
xd = encode_df(csv_data.dropna(), enc)

(749, 6)
(749, 63)
OneHotEncoder(handle_unknown='ignore')


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 0 dimension(s)

In [46]:
csv_data.shape

(153627, 69)

In [45]:
csv_data.isna().sum().sum()

497526

In [47]:
X = csv_data.head()

In [48]:
X.select_dtypes(include=[object])

,GroupNameDe,LastUpdate,Locality,Name,StateShort,StreetAndNr
0,Haus,2016-01-31 12:02:16,Kappel,Immoscout,SO,NaN
1,Haus,2016-02-01 09:07:07,Buchs AG,NabHome,AG,Im Hag 3
2,Haus,2016-02-01 09:52:31,Bassecourt,Homegate,JU,NaN
3,Haus,2016-02-01 09:52:33,Saint-Oyens,Homegate,VD,NaN
4,Haus,2016-02-01 09:54:53,Chambésy,Homegate,GE,NaN


In [38]:
enc = sklearn.preprocessing.OneHotEncoder(handle_unknown='ignore')
enc.fit(X = X)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [39]:
enc.transform(csv_data.tail(1)).toarray()

NotFittedError: This OneHotEncoder instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
enc.inverse_transform(enc.transform(csv_data.tail(1).fillna(method='backfill', axis = 1)).toarray())

In [113]:
def standardise_df(df_, to_ignore = [None]):
    """
    This functions standardises all numbers (based on pandas.api.types.is_numeric_dtype) to standard score.
    -df_: Dataframe to standardise. Type Dataframe.
    -to_ignore: if we don't want to standardise certain columns. Type list.
    returns:
    -df: Standardised Dataframe. All numbers are now standard score. Type Dataframe.
    """
    df = df_.copy()
    assert isinstance(to_ignore, list), "Parameter to_ignore should be a list"
    for col in df.columns:
        if is_numeric_dtype(df[col]) and not col in to_ignore:
            df[col] = (df[col] - df[col].mean()) / df[col].std()
    return df

In [114]:
standardise_df(csv_data_head, to_ignore = ['PurchasePrice'])

,Id,AreaLiving,AreaProperty,BuiltYear,FloorNumber,ForestDensityL,ForestDensityM,ForestDensityS,GroupNameDe,HouseObject,...,gde_private_apartments,gde_social_help_quota,gde_tax,gde_workers_sector1,gde_workers_sector2,gde_workers_sector3,gde_workers_total,location_has_street,location_is_complete,PurchasePrice
0,-1.787453,-0.782780,-0.424949,0.805594,NaN,1.753186,1.757438,1.750298,Haus,NaN,...,-0.322636,0.990816,0.488242,-0.309059,-0.626647,-0.593697,-0.632524,-0.447214,-0.447214,745000.0
1,0.381550,-0.759011,-0.796490,0.415790,NaN,-0.486134,-0.305596,-0.375758,Haus,NaN,...,1.257715,0.968416,-0.434308,-0.545725,1.336207,1.651850,1.505318,1.788854,1.788854,780000.0
2,0.458636,-0.624322,-0.074972,-1.500744,NaN,-0.679231,-0.622637,-0.624251,Haus,NaN,...,0.757792,-0.006254,0.864358,1.779175,0.813531,0.127393,0.536074,-0.447214,-0.447214,570000.0
3,0.459001,0.888946,-0.433243,0.805594,NaN,-0.461079,-0.622637,-0.624251,Haus,NaN,...,-1.238165,-0.682078,0.637269,-0.378666,-0.769536,-0.908943,-0.868902,-0.447214,-0.447214,920000.0
4,0.488266,1.277167,1.729654,-0.526235,NaN,-0.126742,-0.206569,-0.126037,Haus,NaN,...,-0.454705,-1.270899,-1.555561,-0.545725,-0.753555,-0.276603,-0.539967,-0.447214,-0.447214,3950000.0
